In [424]:
import pandas as pd 
from collections import defaultdict
import pickle

In [425]:
train_file = './init_rdc_data/rdc-catalog-train.tsv'

In [426]:
data_train = pd.read_csv(train_file,sep='\t',header=-1)

In [427]:
classes_standard = data_train.iloc[:,1].unique().tolist()

In [428]:
classes_standard[0]

'3292>114>1231'

In [429]:
def tree(): 
    return defaultdict(tree)

In [430]:
base_path = './init_rdc_data/topN_data/8-level-tree/ablstm-tree/predict_out_result_prob.txt.init3-min1-%d'

In [431]:
def load_data(tag,threshold=0.000001,sep='\t'):
    data = pd.read_csv(base_path%tag,sep=sep,header=-1)
    data['label_list'] = data[[i for i in data.columns if i%2==0]].apply(lambda s:tuple(s),axis=1)
    data['prob_list'] = data[[i for i in data.columns.tolist()[:-1] if i%2==1]].apply(lambda s:tuple(s),axis=1)
    data['prob_list_f'] = data['prob_list'].apply(lambda s : [i for i in s if i> threshold])
    data['prob_list_len'] = data['prob_list_f'].apply(lambda s :len(s))
    data['label%d_list_final'%tag] = list(map(lambda x,y: [ str(i)  for i in x[:y]],data['label_list'],data['prob_list_len']))
    data['prob%d_list_final'%tag] = list(map(lambda x,y: x[:y],data['prob_list'],data['prob_list_len']))
    return data[['label%d_list_final'%tag,'prob%d_list_final'%tag]]

In [432]:
data0 = load_data(0)
data1 = load_data(1)
data2 = load_data(2)
data3 = load_data(3)
data4 = load_data(4)
data5 = load_data(5)
data6 = load_data(6)
data7 = load_data(7)

In [433]:
data0.head(2)

,label0_list_final,prob0_list_final
0,[1608],"(1.0,)"
1,[2199],"(1.0,)"


In [434]:
import os 
def add_dicts(dict0,key_list):
    key_len = len(key_list)
    if key_len==1:
        dict0[key_list[0]]
    elif key_len ==2:
        dict0[key_list[0]][key_list[1]]
    elif key_len ==3:
        dict0[key_list[0]][key_list[1]][key_list[2]]
    elif key_len ==4:
        dict0[key_list[0]][key_list[1]][key_list[2]][key_list[3]]
    elif key_len ==5:
        dict0[key_list[0]][key_list[1]][key_list[2]][key_list[3]][key_list[4]]
    elif key_len ==6:
        dict0[key_list[0]][key_list[1]][key_list[2]][key_list[3]][key_list[4]][key_list[5]] 
    elif key_len ==7:
        dict0[key_list[0]][key_list[0]][key_list[0]][key_list[3]][key_list[4]][key_list[5]][key_list[6]]
    return dict0

## 生成树
def generate_data(data_origin_df,output_file):
    data = data_origin_df.copy()
    data.columns = ['title','target']
    data['target_split'] = data['target'].apply(lambda s : s.split('>'))  
    ## 去重
    TiTle = data[['target_split','target']].drop_duplicates('target')
  
    categories = tree()
    for i in TiTle['target_split'].tolist():
        add_dicts(categories,i)

    # 将生成的树保存至Pkl 文件中
    outputs = open(output_file,'wb')
    pickle.dump(categories, outputs)
    outputs.close()
    
    return categories

#generate_data(tree_output_file,output_file)  

In [435]:
#生成树
tree_output_file='./init_rdc_data/category_tree_rdc.pkl'
  

In [436]:
inputs = open(tree_output_file,'rb')
category_tree = pickle.load(inputs)
inputs.close()

In [437]:
def generate_label(category_tree,label_list):
    L = []
    dicts ={}
    for i,j in enumerate(label_list):
        if i == 0:
            L.append(label_list[i])
            dicts = category_tree[j]
        else:
           # print(category_tree[label_list[i-1]].keys())
            if j in dicts.keys():
                L.append(j)
                dicts = dicts[j]
            else:
                #print(i,j)
                break
    #print('L',L)
    return '>'.join(L)

In [438]:
from itertools import permutations,product,combinations

In [439]:
All_Data = pd.concat([data0,data1,data2,data3,data4,data5,data6,data7],axis=1)

In [440]:
All_Data.head(3)

,label0_list_final,prob0_list_final,label1_list_final,prob1_list_final,label2_list_final,prob2_list_final,label3_list_final,prob3_list_final,label4_list_final,prob4_list_final,label5_list_final,prob5_list_final,label6_list_final,prob6_list_final,label7_list_final,prob7_list_final
0,[1608],"(1.0,)",[2320],"(1.0,)",[2173],"(1.0,)","[3813, 2878, 1546]","(0.566875, 0.432849, 0.00027487700000000003)","[2363, 3923, 3606]","(0.941559, 0.0208699, 0.00782667)","[1399, 2517, 2600]","(0.8414959999999999, 0.0734155, 0.0287458)","[1883, 3452, 1729]","(0.349367, 0.138015, 0.08404760000000001)","[4481, 2444]","(0.9999959999999999, 3.5880199999999995e-06)"
1,[2199],"(1.0,)",[4592],"(1.0,)",[12],"(1.0,)","[3609, 2845, 1489]","(0.610422, 0.124628, 0.0947992)","[3796, 1424, 4401]","(0.556242, 0.20908400000000002, 0.075344899999...","[4899, 1391, 229]","(0.9104120000000001, 0.0151095, 0.011604600000...","[1883, 3452, 1046]","(0.546412, 0.0935856, 0.0481574)","[4481, 2444]","(0.9999969999999999, 2.59897e-06)"
2,[2199],"(1.0,)",[4592],"(1.0,)",[12],"(1.0,)","[512, 1500, 98]","(0.849407, 0.0643616, 0.0222667)","[4157, 101, 2787]","(0.817684, 0.17836, 0.00349616)","[2157, 4421, 471]","(0.692923, 0.30696100000000004, 3.28739e-05)","[1883, 3452, 2963]","(0.54779, 0.10340099999999999, 0.0228807)","[4481, 2444]","(0.999999, 1.01321e-06)"


In [441]:
# 计算排列组合
def Permutation_and_Probablity(list_k):
    list_length= len(list_k)
    if list_length==1:
        return list(product(list_k[0]))
    elif list_length==2:
        return list(product(list_k[0],list_k[1]))
    elif list_length==3:
        return list(product(list_k[0],list_k[1],list_k[2]))
    elif list_length==4:
        return list(product(list_k[0],list_k[1],list_k[2],list_k[3]))
    elif list_length==5:
        return list(product(list_k[0],list_k[1],list_k[2],list_k[3],list_k[4]))
    elif list_length==6:
        return list(product(list_k[0],list_k[1],list_k[2],list_k[3],list_k[4],list_k[5]))
    elif list_length==7:
        return list(product(list_k[0],list_k[1],list_k[2],list_k[3],list_k[4],list_k[5],list_k[6]))
    elif list_length==8:
        return list(product(list_k[0],list_k[1],list_k[2],list_k[3],list_k[4],list_k[5],list_k[6]))
    else:
        pass
    

In [442]:
def pading_index(inputs_list):
    # 遇到有 空列表  的 停止，并返回路径的排列组合
    inputs_list = [list(i) for i in inputs_list]
    if [] in inputs_list:
        flag= inputs_list.index([])
        inputs_list=inputs_list[:flag]
    return Permutation_and_Probablity(inputs_list)

In [443]:
import numpy as np

In [444]:
def pading_prob(inputs_list):
    # 将所有的概率/label 从二维数组修改为一维数组
    inputs_list = [list(i) for i in inputs_list]
    if [] in inputs_list:
        flag= inputs_list.index([])
        inputs_list=inputs_list[:flag]
    prob_list=[]
    for i in inputs_list:
        prob_list.extend(i)
    return prob_list

In [445]:
# 将概率/label横向拼接

In [446]:
All_Data['prob_all'] = All_Data[['prob%d_list_final'%(i-1) for i in range(1,9)]].apply(lambda s: tuple(s),axis=1)

In [447]:
All_Data['label_all']= All_Data[['label%d_list_final'%(i-1) for i in range(1,9)]].apply(lambda s: tuple(s),axis=1)

In [492]:
All_Data['all_path_refer'] = All_Data['label_all'].apply(lambda s: pading_index(s))

In [ ]:
#依靠类目树查找最大搜索路径

In [501]:
All_Data['tree_paths'] = All_Data['all_path_refer'].apply(lambda s :','.join(list(pd.Series([generate_label(category_tree,i) for i in s]).unique())))

In [504]:
#计算可拼接成的路径数

In [505]:
All_Data['path_lenth']= All_Data['tree_paths'].apply(lambda s:len(s.split(',')))

In [506]:
# 做成一维列表 用于后续生成 label-prob字典

In [507]:
All_Data['prob_all_pads'] = All_Data['prob_all'].apply(lambda s: pading_prob(s))

In [508]:
All_Data['label_all_pads']= All_Data['label_all'].apply(lambda s: pading_prob(s))

In [509]:
All_Data['label_prob_dict'] =list(map(lambda x,y:dict(zip(x,y)), All_Data['label_all_pads'],All_Data['prob_all_pads']))

In [510]:
All_Data.head(2)

,label0_list_final,prob0_list_final,label1_list_final,prob1_list_final,label2_list_final,prob2_list_final,label3_list_final,prob3_list_final,label4_list_final,prob4_list_final,...,all_path_refer,tree_paths,path_lenth,prob_all_pads,label_all_pads,label_prob_dict,selected_label,selected_label_depth,selected_label_depth_str,selected_label_str
0,[1608],"(1.0,)",[2320],"(1.0,)",[2173],"(1.0,)","[3813, 2878, 1546]","(0.566875, 0.432849, 0.00027487700000000003)","[2363, 3923, 3606]","(0.941559, 0.0208699, 0.00782667)",...,"[(1608, 2320, 2173, 3813, 2363, 1399, 1883), (...","1608>2320>2173>3813,1608>2320>2173>2878,1608>2...",3,"[1.0, 1.0, 1.0, 0.566875, 0.432849, 0.00027487...","[1608, 2320, 2173, 3813, 2878, 1546, 2363, 392...","{'1608': 1.0, '2320': 1.0, '2173': 1.0, '3813'...","([1608>2320>2173>2878, 1608>2320>2173>3813], [...","([1608>2320>2173>2878, 1608>2320>2173>3813], [...","(['1608>2320>2173>2878', '1608>2320>2173>3813'...","(['1608>2320>2173>2878', '1608>2320>2173>3813'..."
1,[2199],"(1.0,)",[4592],"(1.0,)",[12],"(1.0,)","[3609, 2845, 1489]","(0.610422, 0.124628, 0.0947992)","[3796, 1424, 4401]","(0.556242, 0.20908400000000002, 0.075344899999...",...,"[(2199, 4592, 12, 3609, 3796, 4899, 1883), (21...",2199>4592>12,1,"[1.0, 1.0, 1.0, 0.610422, 0.124628, 0.0947992,...","[2199, 4592, 12, 3609, 2845, 1489, 3796, 1424,...","{'2199': 1.0, '4592': 1.0, '12': 1.0, '3609': ...","([2199>4592>12], [1.0])","([2199>4592>12], [1.0])","(['2199>4592>12'], [1.0])","(['2199>4592>12'], [1.0])"


In [511]:
# 计算概率连乘分数 并返回分数最大的那个

In [512]:
def generate_scores(paths,dicts,max_depth=False):
    paths_list=paths.split(',')
    #print('paths,dicts',paths,dicts)
    #print('paths_list',paths_list)
    if max_depth==True:
        paths_lists_tmp = []
        lenth_list = [len(i.split('>')) for i in paths_list ]
        max_lenth= max(lenth_list)
        for i in paths_list:
            if len(i.split('>'))==max_lenth:
                paths_lists_tmp.append(i)
            else:
                continue
        paths_list= paths_lists_tmp
    scores_list = []
    #print('paths_list',paths_list)
    paths_list=list(set(paths_list)&set(classes_standard))
    #print('paths_list',paths_list)
    for path_base in paths_list:  
        #print('path_base',path_base)
        scores_list.append(np.mean([dicts[i] for i in path_base.split('>')]))
    
    score_list_new = sorted(scores_list)[-10:]
    index_list = [scores_list.index(i) for i in scores_list]
    return [paths_list[i] for i in index_list],score_list_new

In [513]:
#计算权重最大的路径

In [514]:
All_Data['selected_label'] = list(map(lambda x,y:generate_scores(x,y),All_Data['tree_paths'],All_Data['label_prob_dict']))

In [515]:
#计算深度最大中权重最大的路径

In [516]:
All_Data['selected_label_depth'] = list(map(lambda x,y:generate_scores(x,y,True),All_Data['tree_paths'],All_Data['label_prob_dict']))

In [517]:
All_Data.head(2)

,label0_list_final,prob0_list_final,label1_list_final,prob1_list_final,label2_list_final,prob2_list_final,label3_list_final,prob3_list_final,label4_list_final,prob4_list_final,...,all_path_refer,tree_paths,path_lenth,prob_all_pads,label_all_pads,label_prob_dict,selected_label,selected_label_depth,selected_label_depth_str,selected_label_str
0,[1608],"(1.0,)",[2320],"(1.0,)",[2173],"(1.0,)","[3813, 2878, 1546]","(0.566875, 0.432849, 0.00027487700000000003)","[2363, 3923, 3606]","(0.941559, 0.0208699, 0.00782667)",...,"[(1608, 2320, 2173, 3813, 2363, 1399, 1883), (...","1608>2320>2173>3813,1608>2320>2173>2878,1608>2...",3,"[1.0, 1.0, 1.0, 0.566875, 0.432849, 0.00027487...","[1608, 2320, 2173, 3813, 2878, 1546, 2363, 392...","{'1608': 1.0, '2320': 1.0, '2173': 1.0, '3813'...","([1608>2320>2173>2878, 1608>2320>2173>3813], [...","([1608>2320>2173>2878, 1608>2320>2173>3813], [...","(['1608>2320>2173>2878', '1608>2320>2173>3813'...","(['1608>2320>2173>2878', '1608>2320>2173>3813'..."
1,[2199],"(1.0,)",[4592],"(1.0,)",[12],"(1.0,)","[3609, 2845, 1489]","(0.610422, 0.124628, 0.0947992)","[3796, 1424, 4401]","(0.556242, 0.20908400000000002, 0.075344899999...",...,"[(2199, 4592, 12, 3609, 3796, 4899, 1883), (21...",2199>4592>12,1,"[1.0, 1.0, 1.0, 0.610422, 0.124628, 0.0947992,...","[2199, 4592, 12, 3609, 2845, 1489, 3796, 1424,...","{'2199': 1.0, '4592': 1.0, '12': 1.0, '3609': ...","([2199>4592>12], [1.0])","([2199>4592>12], [1.0])","(['2199>4592>12'], [1.0])","(['2199>4592>12'], [1.0])"


In [518]:
All_Data['selected_label_depth_str'] = All_Data['selected_label_depth'].apply(lambda s:str(s))

In [519]:
All_Data['selected_label_str']= All_Data['selected_label'].apply(lambda s:str(s))

In [520]:
output_path = './init_rdc_data/topN_data/8-level-tree/'

In [521]:
All_Data[['selected_label_depth_str']].to_csv(os.path.join(output_path,'level_tree_max_depth_1.tsv'),sep='\t',header=None,index=False)

In [522]:
All_Data[['selected_label_str']].to_csv(os.path.join(output_path,'levle_tree_max_prob_1.tsv'),sep='\t',header=None,index=False)